In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from io import StringIO
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn import model_selection
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize

from sklearn.metrics import accuracy_score
from sklearn.linear_model import Perceptron
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.mixture import GaussianMixture
from sklearn.tree import export_graphviz
from IPython.display import Image
import graphviz
import six
import sys
sys.modules['sklearn.externals.six'] = six
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
from sklearn.metrics import confusion_matrix
sns.set_style("darkgrid")
import os
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.naive_bayes import GaussianNB 
from sklearn.decomposition import PCA  
import math
import time
from sklearn.preprocessing import StandardScaler

from scipy.stats import sem

plt.rcParams['figure.dpi'] = 600

In [ ]:
current_directory = os.getcwd()
current_directory

In [ ]:
hs = pd.read_csv('hDRG_py.csv')
hs

In [ ]:
# Tuning the Dataset

hs = pd.read_csv('hDRG_py.csv')
hs['multi'] = hs['APs'].apply(lambda x: '0' if x==1 else '1')
hs.drop("APs", axis=1, inplace=True)
hs.drop("Cell ID", axis=1, inplace=True)
hs.drop("mAHP", axis=1, inplace=True)

#Intentionally Dropped
hs.drop("R_slope", axis=1, inplace=True)
hs.drop("D_slope", axis=1, inplace=True)
hs.drop("Sag", axis=1, inplace=True)
hs.drop("Norm_rheo", axis=1, inplace=True)
hs.drop("Decay", axis=1, inplace=True)
hs.drop("Amp", axis=1, inplace=True)
hs.drop("Rise", axis=1, inplace=True)

display(hs)

In [ ]:
hs = pd.DataFrame(hs)

# Columns to normalize
columns_to_normalize = ['Size', 'Cap', 'Rebound', 'SA_rest', 'SA_45', 'RMP', 'Rin', 'Rheo', 'FSL', 'Sag_R', 'Peak', 'Thresh', 'HW', 'fAHP']

# Extract the columns to be normalized
subset_hs = hs[columns_to_normalize]

# Normalize the subset of columns along rows (axis=1)
normalized_subset = normalize(subset_hs, norm='l2', axis=1)

# Create a DataFrame with the normalized subset and the non-normalized columns
normalized_hs = pd.DataFrame(normalized_subset, columns=columns_to_normalize)

# Concatenate the normalized subset DataFrame with the non-normalized columns
result_hs = pd.concat([normalized_hs, hs.drop(columns=columns_to_normalize)], axis=1)

print("Original DataFrame:")
print(hs)

print("\nNormalized DataFrame (specific columns only):")
print(result_hs)


In [ ]:
# Separate features (X) and target variable (y)
X = result_hs[['Size', 'Cap', 'Rebound', 'SA_rest', 'SA_45', 'RMP', 'Rin', 'Rheo', 'FSL', 'Sag_R', 'Peak', 'Thresh', 'HW', 'fAHP']]
y = hs['multi']

# Initialize PCA with 3 components
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X)

# Create a new DataFrame with PCA components and target variable
pca_hs = pd.DataFrame(data=X_pca, columns=['PC1', 'PC2', 'PC3'])
pca_hs['multi'] = y

# Initialize 3D plot
fig = plt.figure(figsize=(14, 14))
ax = fig.add_subplot(111, projection='3d')

# Define colors for each category (0: 'red', 1: 'green')
#colors = {0: 'r', 1: 'g'}

# Plot each category separately
for category in pca_hs['multi'].unique():
    subset = pca_hs[pca_hs['multi'] == category]
    #color = colors[category]  # Use integer keys to access colors
    ax.scatter(subset['PC1'], subset['PC2'], subset['PC3'], label=str(category), s=100)

# Draw lines representing principal axes scaled by eigenvalues
#for length, vector in zip(pca.explained_variance_, pca.components_):
    #scaled_vector = vector * 30 * length  # Scale vector by 3 times the eigenvalue for visualization
   # ax.plot([0, scaled_vector[0]], [0, scaled_vector[1]], [0, scaled_vector[2]], color='black')

# Set background color to white
ax.set_facecolor('white')


# Set labels and title
ax.set_xlabel('Principal Component 1', fontsize=16)
ax.set_ylabel('Principal Component 2', fontsize=16)
ax.set_zlabel('Principal Component 3', fontsize=16)
ax.set_title('3D PCA Plot', fontsize=20)


# Add legend
ax.legend(title='Category')

# Show plot
plt.savefig('PCAnorm.png')
plt.show()


In [ ]:
# Plot the explained variance ratio for the principal components
plt.figure(figsize=(8, 6))
plt.bar(range(1, pca.n_components_ + 1), pca.explained_variance_ratio_, alpha=0.8, align='center')
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')
plt.title('Explained Variance Ratio of Principal Components')
plt.xticks(range(1, pca.n_components_ + 1))
plt.grid(True)
plt.savefig('PCAnormComp.png')
plt.show()

In [ ]:
# Extract explained variance values
explained_variance = pca.explained_variance_
explained_variance_ratio = pca.explained_variance_ratio_
cumulative_explained_variance_ratio = np.cumsum(explained_variance_ratio)

# Display explained variance values
for i in range(len(explained_variance)):
    print(f"Principal Component {i + 1}: Explained Variance = {explained_variance[i]:.4f}, Explained Variance Ratio = {explained_variance_ratio[i]:.4f}")

In [ ]:
# Get the PCA loadings (eigenvectors)
loadings = pd.DataFrame(pca.components_.T, columns=['PC1', 'PC2', 'PC3'], index=X.columns)

# Display feature importance (absolute PCA loadings)
print("Feature Importance (Absolute PCA Loadings):")
print(loadings.abs())

# Sort features based on absolute loadings (for PC1, PC2, or PC3)
sorted_loadings = loadings.abs().sort_values(by='PC1', ascending=False)

# Plotting feature importance
plt.figure(figsize=(10, 6))
plt.barh(sorted_loadings.index, sorted_loadings['PC1'], color='skyblue')
plt.xlabel('Absolute PCA Loadings')
plt.title('Feature Importance - PC1')
plt.gca().invert_yaxis()  # Invert y-axis to display most important features on top
plt.savefig('PC1PCAnorm.png')
plt.show()


In [ ]:
# Sort features based on absolute loadings (for PC1, PC2, or PC3)
sorted_loadings = loadings.abs().sort_values(by='PC2', ascending=False)

# Plotting feature importance
plt.figure(figsize=(10, 6))
plt.barh(sorted_loadings.index, sorted_loadings['PC2'], color='skyblue')
plt.xlabel('Absolute PCA Loadings')
plt.title('Feature Importance - PC2')
plt.gca().invert_yaxis()  # Invert y-axis to display most important features on top
plt.savefig('PC2PCAnorm.png')
plt.show()


In [ ]:
# Sort features based on absolute loadings (for PC1, PC2, or PC3)
sorted_loadings = loadings.abs().sort_values(by='PC3', ascending=False)

# Plotting feature importance
plt.figure(figsize=(10, 6))
plt.barh(sorted_loadings.index, sorted_loadings['PC3'], color='skyblue')
plt.xlabel('Absolute PCA Loadings')
plt.title('Feature Importance - PC3')
plt.gca().invert_yaxis()  # Invert y-axis to display most important features on top
plt.savefig('PC3PCAnorm.png')
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Assuming 'result_hs' is your DataFrame containing the data

# Separate features (X) and target variable (y)
X = result_hs[['Size', 'Cap', 'RMP', 
              'Rin', 'Rheo', 'FSL', 'Sag_R', 'Peak', 'Thresh', 'HW', 'fAHP']]
y = result_hs['multi']

# Initialize PCA with 2 components
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

# Create a new DataFrame with PCA components and target variable
pca_df = pd.DataFrame(data=X_pca, columns=['PC1', 'PC2'])
pca_df['multi'] = y

# Plotting the 2D PCA
plt.figure(figsize=(10, 8))
for category in pca_df['multi'].unique():
    subset = pca_df[pca_df['multi'] == category]
    plt.scatter(subset['PC1'], subset['PC2'], label=str(category))

# Draw lines representing principal axes scaled by eigenvalues
for length, vector in zip(pca.explained_variance_, pca.components_[:2]):  # Consider only first two components
    scaled_vector = vector * 15 * length  # Scale vector by 3 times the eigenvalue for visualization
    plt.plot([0, scaled_vector[0]], [0, scaled_vector[1]], color='black')

# Set labels and title
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('2D PCA Plot')

# Add legend
plt.legend(title='Category')

# Show plot
plt.grid(True)
plt.show()


In [ ]:
# Display feature importance (absolute PCA loadings)
print("Feature Importance (Absolute PCA Loadings):")
print(loadings.abs())